<a href="https://colab.research.google.com/github/zhouxiaoqi0229/note/blob/main/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pwd


/content


In [ ]:
import os
os.chdir('./drive/MyDrive/Colab Notebooks/movie_recommender')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split#数据集划分
import numpy as np
from collections import Counter#用于统计字符出现的次数
import tensorflow as tf

import pickle #提供保存数据在本地的方法
import re #正则表达式
from tensorflow.python.ops import math_ops 
from urllib.request import urlretrieve#将url表示的网络对象复制到本地
from os.path import isfile,isdir#判断是否存在文件file 文件夹dir
from tqdm import tqdm  #Tqdm 是一个快速，可扩展的Python进度条，可以在 Python 长循环中添加一个进度提示信息，用户只需要封装任意的迭代器 tqdm(iterator)
import zipfile #用来做zip格式的压缩与解压缩
import hashlib #hash 或者MD5加密



In [ ]:
#数据查看

#用户数据
users_title = ['UserID', 'Gender', 'Age', 'OccupationID', 'Zip-code']
users = pd.read_csv('./users.dat',sep='::', header=None, names=users_title, engine = 'python')
users.info



In [ ]:
#电影数据
movies_title = ['MovieID', 'Title', 'Genres']
movies = pd.read_csv('./movies.dat', sep='::', header=None, names=movies_title, engine = 'python')
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
#评分数据
ratings_title = ['UserID','MovieID', 'Rating', 'timestamps']
ratings = pd.read_csv('./ratings.dat', sep='::', header=None, names=ratings_title, engine = 'python')
ratings.head()

,UserID,MovieID,Rating,timestamps
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


**处理数据**


*   userID movieId occuption不变
*   性别 0,1 

*   年龄 0-6
*   类别;是分类字段，要转成数字。首先将Genres中的类别转成字符串到数字的字典，然后再将每个电影的Genres字段转成数字列表，因为有些电影是多个Genres的组合。


*   Title字段：处理方式跟Genres字段一样，首先创建文本到数字的字典，然后将Title中的描述转成数字的列表。另外Title中的年份也需要去掉
*   Genres和Title字段需要将长度统一，这样在神经网络中方便处理。空白部分用‘< PAD >’对应的数字填充。







In [ ]:
#数据预处理

def load_data():
  #修改user数据集
  users_title = ['UserID', 'Gender', 'Age', 'JobID', 'Zip-code']
  users = pd.read_csv('./users.dat',sep='::', header=None, names=users_title, engine = 'python')
  users = users.filter(regex='UserID|Gender|Age|JobID')
  users_orig = users.values
  users['Gender'] = users['Gender'].map({'F':0, 'M':1})#性别映射

  age_map = {val:ii for ii, val in enumerate(set(users['Age']))}
  users['Age'] = users['Age'].map(age_map)#年龄映射为0-6
  #修改movie数据集
  movies_title = ['MovieID', 'Title', 'Genres']
  movies = pd.read_csv('./movies.dat', sep='::', header=None, names=movies_title, engine = 'python')
  movies_orig = movies.values

  #去掉标题中的年份
  pattern = re.compile(r'^(.*)\((\d+)\)$')
  title_map = {val:pattern.match(val).group(1) for ii,val in enumerate(set(movies['Title']))}
  movies['Title'] = movies['Title'].map(title_map)

  #电影类型转换为数字字典
  genres_set = set()
  for val in movies['Genres'].str.split('|'):
    genres_set.update(val)
  genres_set.add('<PAD>')

  genres2int = {val:ii for ii, val in enumerate(genres_set)} #每种类型到一个数字的映射
  #eg: a|b|c:1|2|3这种类型的一个映射
  genres_map = {val:[genres2int[row] for row in val.split('|')] for ii,val in enumerate(set(movies['Genres']))} 
  #将类型补齐
  for key in genres_map:
    for cnt in range(max(genres2int.values())-len(genres_map[key])):
      genres_map[key].insert(len(genres_map[key])+cnt,genres2int['<PAD>'])
  movies['Genres'] = movies['Genres'].map(genres_map)
  movies.head(10)

  #电影Title转数字字典
  title_set = set()#所有电影名称涉及到的单词
  for val in movies['Title'].str.split():
    title_set.update(val)
  title_set.add('<PAD>')

  title_count = 15
  title2int = {val:ii for ii, val in enumerate(title_set)}#每个单词映射成一个数字
  title_map = {val:[title2int[row] for row in val.split()] for ii,val in enumerate(set(movies['Title']))}
  for key in title_map:
    for cnt in range(title_count - len(title_map[key])):
      title_map[key].insert(len(title_map[key]) + cnt,title2int['<PAD>'])
    
  movies['Title'] = movies['Title'].map(title_map)
  movies.head(5)

  #读取评分数据集
  ratings_title = ['UserID','MovieID', 'ratings', 'timestamps']
  ratings = pd.read_csv('./ratings.dat', sep='::', header=None, names=ratings_title, engine = 'python')
  ratings = ratings.filter(regex='UserID|MovieID|ratings')#不要时间戳

  #合并三个表
  data = pd.merge(pd.merge(ratings, users), movies)#1000209 rows × 8 columns
  #分成两张表
  target_fields = ['ratings']

  features_pd,targets_pd = data.drop(target_fields,axis=1),data[target_fields]# 一个表没有评分 另一个表只有评分
  features = features_pd.values #将dataframe转变为array
  targets_values = targets_pd.values

  return title_count, title_set, genres2int, features, targets_values, ratings, users, movies, data, movies_orig, users_orig

    



*   title_count:title字段的长度（15）

*   title_set：title文本的集合

*   genres2int：电影类型转数字的字典

*   features：是输入X
*   ratings：评分数据集的Pandas对象


*   movies_orig：没有做数据处理的原始电影数据


*   users_orig：没有做数据处理的原始用户数据




In [ ]:
title_count, title_set, genres2int, features, targets_values, ratings, users, movies, data, movies_orig, users_orig = load_data()

pickle.dump((title_count, title_set, genres2int, features, targets_values, ratings, users, movies, data, movies_orig, users_orig), open('preprocess.p', 'wb'))

In [ ]:
#辅助函数
def save_params(params):
  '''
  save params to filr
  '''
  pickle.dump(params,open('params.p','wb'))
def load_params():
    """
    Load parameters from file
    """
  return pickle.load(open('params.p', mode='rb'))


In [ ]:
import numpy as np
student = np.array([('name','S20'), ('age', 'i1'), ('marks', 'f4')]) 
student.take(1,0)

array(['age', 'i1'], dtype='<U5')

In [ ]:
#嵌入矩阵的维度
embed_dim = 32
#用户ID个数
uid_max = max(features.take(0,1)) + 1 # 6040+1
#性别个数
gender_max = max(features.take(2,1)) + 1 # 1 + 1 = 2
#年龄类别个数
age_max = max(features.take(3,1)) + 1 # 6 + 1 = 7
#职业个数
job_max = max(features.take(4,1)) + 1# 20 + 1 = 21

#电影ID个数
movie_id_max = max(features.take(1,1)) + 1 # 3952
#电影类型个数
movie_categories_max = max(genres2int.values()) + 1 # 18 + 1 = 19
#电影名单词个数
movie_title_max = len(title_set) # 5216
#电影名长度
sentences_size = title_count # = 15

#对电影类型嵌入向量做加和操作的标志
combiner = "sum"
#文本卷积滑动窗口，分别滑动2, 3, 4, 5个单词
window_sizes = {2, 3, 4, 5}
#文本卷积核数量
filter_num = 8

#电影ID转下标的字典，数据集中电影ID跟下标不一致，比如第5行的数据电影ID不一定是5
movieid2idx = {val[0]:i for i, val in enumerate(movies.values)}
movieid2idx

In [ ]:
#超参数设置
num_epochs= 5
batch_size =256
dropout_keep = 0.5
learning_rate = 0.0001
show_every_n_batches = 20

save_dir = './'

In [ ]:
# #定义输入的占位符
def get_inputs():
  uid = tf.keras.layers.Input(shape=(1,), dtype='int32', name='uid')  
  user_gender = tf.keras.layers.Input(shape=(1,), dtype='int32', name='user_gender')  
  user_age = tf.keras.layers.Input(shape=(1,), dtype='int32', name='user_age') 
  user_job = tf.keras.layers.Input(shape=(1,), dtype='int32', name='user_job')

  movie_id = tf.keras.layers.Input(shape=(1,), dtype='int32', name='movie_id') 
  movie_categories = tf.keras.layers.Input(shape=(18,), dtype='int32', name='movie_categories') 
  movie_titles = tf.keras.layers.Input(shape=(15,), dtype='int32', name='movie_titles') 
  return uid, user_gender, user_age, user_job, movie_id, movie_categories, movie_titles

tf.keras.Input函数用于向模型中输入数据，并指定数据的形状、数据类型等信息

keras.layers.Embedding(input_dim,output_dim,input_length)


*   input_dim:词汇表的维度（有多少个不同词汇）
*   output_dim:嵌入词空间的维度（每个词嵌入为多少维度）


*   input_length:输入语句的长度（每一个词汇多少维）




In [ ]:
#test  embedding
from tensorflow import keras
import numpy as np

data = np.array([[0,1,2],[3,4,5]])
print(len(data))
emb = keras.layers.Embedding(input_dim=6, output_dim=4, input_length=3) # 只有0,1所以input_dim是2
emb(data)


<tf.Tensor: shape=(2, 3, 4), dtype=float32, numpy=
array([[[ 0.0439744 , -0.02778723,  0.01964632, -0.02765288],
        [-0.00518662,  0.03885773, -0.01417477,  0.03181381],
        [-0.04332082, -0.01849406, -0.04606277, -0.0005514 ]],

       [[-0.0499388 ,  0.00722983, -0.04274812, -0.0004092 ],
        [-0.0242069 ,  0.03826846, -0.00732299, -0.02551873],
        [-0.03418063, -0.00546832,  0.02819309, -0.04676614]]],
      dtype=float32)>

In [ ]:
#定义用户的嵌入矩阵
def get_user_embedding(uid, user_gender, user_age, user_job):
  uid_embed_layer = tf.keras.layers.Embedding(uid_max, embed_dim, input_length=1, name='uid_embed_layer')(uid) 
  gender_embed_layer = tf.keras.layers.Embedding(gender_max, embed_dim // 2, input_length=1, name='gender_embed_layer')(user_gender)
  age_embed_layer = tf.keras.layers.Embedding(age_max, embed_dim // 2, input_length=1, name='age_embed_layer')(user_age)
  job_embed_layer = tf.keras.layers.Embedding(job_max, embed_dim // 2, input_length=1, name='job_embed_layer')(user_job)
  return uid_embed_layer, gender_embed_layer, age_embed_layer, job_embed_layer


```
tf.keras.layers.Dense
(
    units,
    activation=None,
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)
```
tf.keras.layers.Dense相当于在全连接层中添加一个层

units:输出维度的大小,这一层多少神经元



In [ ]:
#将User的嵌入矩阵一起全连接生成User的特征
def get_user_feature_layer(uid_embed_layer, gender_embed_layer, age_embed_layer, job_embed_layer):
    #第一层全连接
  uid_fc_layer = tf.keras.layers.Dense(embed_dim, name="uid_fc_layer", activation='relu')(uid_embed_layer)
  gender_fc_layer = tf.keras.layers.Dense(embed_dim, name="gender_fc_layer", activation='relu')(gender_embed_layer)
  age_fc_layer = tf.keras.layers.Dense(embed_dim, name="age_fc_layer", activation='relu')(age_embed_layer)
  job_fc_layer = tf.keras.layers.Dense(embed_dim, name="job_fc_layer", activation='relu')(job_embed_layer)

    #第二层全连接
  user_combine_layer = tf.keras.layers.concatenate([uid_fc_layer, gender_fc_layer, age_fc_layer, job_fc_layer], 2)  #(?, 1, 128)
  user_combine_layer = tf.keras.layers.Dense(200, activation='tanh')(user_combine_layer)  #(?, 1, 200)

  user_combine_layer_flat = tf.keras.layers.Reshape([200], name="user_combine_layer_flat")(user_combine_layer)##这一步不太明白
  return user_combine_layer, user_combine_layer_flat




In [ ]:
#定义Movie Id的嵌入矩阵
def get_movie_id_embed_layer(movie_id):
  movie_id_embed_layer = tf.keras.layers.Embedding(movie_id_max,embed_dim,input_length=1,name='movie_id_embed_layer')(movie_id)
  return movie_id_embed_layer

In [ ]:
#合并电影类型的多个嵌入向量
def get_movie_categories_embed_layers(movie_categories):
  movie_categories_embed_layer = tf.keras.layers.Embedding(movie_categories_max,embed_dim,input_length=18,name='movie_categories_embed_layer')(movie_categories)
  movie_categories_embed_layer = tf.keras.layers.Lambda(lambda layer:tf.reduce_sum(layer,axis=1,keepdims=True))(movie_categories_embed_layer)#每一行求一个和(
  return movie_categories_embed_layer


In [ ]:
#title 的文本卷积网络实现
def get_movie_cnn_layer(movie_titles):

    #从嵌入矩阵中得到电影名对应的各个单词的嵌入向量
  movie_title_embed_layer = tf.keras.layers.Embedding(movie_title_max, embed_dim, input_length=15, name='movie_title_embed_layer')(movie_titles)
  sp=movie_title_embed_layer.shape
  movie_title_embed_layer_expand = tf.keras.layers.Reshape([sp[1], sp[2], 1])(movie_title_embed_layer)
    #对文本嵌入层使用不同尺寸的卷积核做卷积和最大池化
  pool_layer_lst = []
  for window_size in window_sizes:
      conv_layer = tf.keras.layers.Conv2D(filter_num, (window_size, embed_dim), 1, activation='relu')(movie_title_embed_layer_expand)
      maxpool_layer = tf.keras.layers.MaxPooling2D(pool_size=(sentences_size - window_size + 1 ,1), strides=1)(conv_layer)
      pool_layer_lst.append(maxpool_layer)
    #Dropout层
  pool_layer = tf.keras.layers.concatenate(pool_layer_lst, 3, name ="pool_layer")  
  max_num = len(window_sizes) * filter_num
  pool_layer_flat = tf.keras.layers.Reshape([1, max_num], name = "pool_layer_flat")(pool_layer)

  dropout_layer = tf.keras.layers.Dropout(dropout_keep, name = "dropout_layer")(pool_layer_flat)
  return pool_layer_flat, dropout_layer




```
tf.keras.layers.Conv2D(
    filters, kernel_size, strides=(1, 1), padding='valid', data_format=None,
    dilation_rate=(1, 1), activation=None, use_bias=True,
    kernel_initializer='glorot_uniform', bias_initializer='zeros',
    kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
    kernel_constraint=None, bias_constraint=None, **kwargs
)
```


*   filters:卷积核的个数
*   kernel_size:卷积核尺寸


*   strides：滑动步长
*   padding：补全策略，


In [ ]:
#将movie各个层一起做全连接
def get_movie_feature_layer(movie_id_embed_layer,movie_categories_embed_layer,dropout_layer):
  #第一层全连接
  movie_id_fc_layer = tf.keras.layers.Dense(embed_dim, name="movie_id_fc_layer", activation='relu')(movie_id_embed_layer)
  movie_categories_fc_layer = tf.keras.layers.Dense(embed_dim,name="movie_categories_fc_layer",activation='relu')(movie_categories_embed_layer)

  #第二层全连接
  movie_combine_layer = tf.keras.layers.concatenate([movie_id_fc_layer,movie_categories_fc_layer,dropout_layer], 2)
  movie_combine_layer = tf.keras.layers.Dense(200, activation='tanh')(movie_combine_layer)
  movie_combine_layer_flat = tf.keras.layers.Reshape([200],name="movie_combine_layer_flat")(movie_combine_layer)

  return movie_combine_layer,movie_combine_layer_flat




In [ ]:
def get_batches(Xs,ys,batch_size):
    for start in range(0,len(Xs),batch_size):
      end = min(start+batch_size,len(Xs))
      yield Xs[start:end],ys[start:end]

In [ ]:
#构建计算图
import tensorflow as tf
import datetime
from tensorflow import keras
from tensorflow.python.ops import summary_ops_v2
import time

MODEL_DIR= "./models"

class mv_network(object):
  def __init__(self,batch_szie = 256):
    self.batch_size=batch_size
    self.best_loss = 9999
    self.losses={'train':[],'test':[]}

    #获取占位符
    uid, user_gender, user_age, user_job, movie_id, movie_categories, movie_titles = get_inputs()
    #d得到用户的4个嵌入向量
    uid_embed_layer,gender_embed_layer,age_embed_layer,job_embed_layer = get_user_embedding(uid,user_gender,user_age,user_job)
    print(uid_embed_layer)
    #得到用户特征
    user_combine_layer,user_combine_layer_flat = get_user_feature_layer(uid_embed_layer,gender_embed_layer,age_embed_layer,job_embed_layer)
    
    #获取电影ID的嵌入特征
    movie_id_embed_layer = get_movie_id_embed_layer(movie_id)
    #获取电影类型的嵌入向量
    movie_categories_embed_layer = get_movie_categories_embed_layers(movie_categories)

    #获取电影名的特征向量
    pool_layer_flat, dropout_layer = get_movie_cnn_layer(movie_titles)
    #得到电影特征

    movie_combine_layer,movie_combine_layer_flat = get_movie_feature_layer(movie_id_embed_layer,movie_categories_embed_layer,dropout_layer)
    #计算出评分
    #将用户特征与电影特征做矩阵乘法得到预测评分的方案
    inference = tf.keras.layers.Lambda(lambda layer:tf.reduce_sum(layer[0] * layer[1],axis=1),name='inference')((user_combine_layer_flat,movie_combine_layer_flat))
    inference = tf.keras.layers.Lambda(lambda layer:tf.expand_dims(layer,axis=1))(inference)
    self.model = tf.keras.Model(
        inputs=[uid,user_gender,user_age,user_job,movie_id,movie_categories,movie_titles],
        outputs=inference
        )
    self.model.summary()
    self.optimizer = tf.keras.optimizers.Adam(learning_rate)
    self.ComputeLoss = tf.keras.losses.MeanSquaredError()
    self.ComputeMetrics = tf.keras.metrics.MeanAbsoluteError()
    
    if tf.io.gfile.exists(MODEL_DIR):
      pass
    else:
      tf.io.gfile.makedirs(MODEL_DIR)
    
    train_dir = os.path.join(MODEL_DIR,'summaries','train')
    test_dir = os.path.join(MODEL_DIR,'summaries','eval')

    checkpoint_dir = os.path.join(MODEL_DIR,'checkpoints')
    self.checkpoint_prefix = os.path.join(checkpoint_dir,'ckpt')
    self.checkpoint = tf.train.Checkpoint(model=self.model,optimizer=self.optimizer)
    self.checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

  def compute_loss(self,labels,logits):
    return tf.reduce_mean(tf.keras.losses.mse(labels,logits))
  def compute_metrics(self,labels,logits):
    return tf.keras.metrics.mae(labels,logits)

  @tf.function
  def train_step(self,x,y):
    #记录用于计算损失的操作，以便可以计算损失相对于变量的梯度
    with tf.GradientTape() as tape:
      logits = self.model([x[0],
                  x[1],
                  x[2],
                  x[3],
                  x[4],
                  x[5],
                  x[6]],training=True)
      loss = self.ComputeLoss(y,logits)
      self.ComputeMetrics(y,logits)
    grads = tape.gradient(loss,self.model.trainable_variables)
    self.optimizer.apply_gradients(zip(grads,self.model.trainable_variables))
    return loss, logits

  def training(self, features, targets_values, epochs=5, log_freq=50):
    for epoch_i in range(epochs):
      train_X,test_X,train_y,test_y = train_test_split(features,targets_values,test_size=0.2,random_state=0)
      train_batches = get_batches(train_X,train_y,self.batch_size)
      batch_num = (len(train_X)//self.batch_size)
      train_start = time.time()
      if True:
        start = time.time()#当前时间的时间戳
        avg_loss = tf.keras.metrics.Mean('loss',dtype=tf.float32)
        for batch_i in range(batch_num):
          x, y = next(train_batches)
          categories = np.zeros([self.batch_size,18])
          for i in range(self.batch_size):
            categories[i] = x.take(6,1)[i]

          titles = np.zeros([self.batch_size, sentences_size])
          for i in range(self.batch_size):
            titles[i] = x.take(5,1)[i]
          loss,logits = self.train_step([np.reshape(x.take(0,1),[self.batch_size,1]).astype(np.float32),
                        np.reshape(x.take(2, 1), [self.batch_size, 1]).astype(np.float32),
                                                    np.reshape(x.take(3, 1), [self.batch_size, 1]).astype(np.float32),
                                                    np.reshape(x.take(4, 1), [self.batch_size, 1]).astype(np.float32),
                                                    np.reshape(x.take(1, 1), [self.batch_size, 1]).astype(np.float32),
                                         categories.astype(np.float32),
                                                    titles.astype(np.float32)], np.reshape(y, [self.batch_size, 1]).astype(np.float32))
          

          avg_loss(loss)
          self.losses['train'].append(loss)
          if tf.equal(self.optimizer.iterations % log_freq,0):
            rate = log_freq / (time.time()-start)
            print('Step #{}\tEpoch {:>3} Batch {:>4}/{}   Loss: {:0.6f} mae: {:0.6f} ({} steps/sec)'.format(
                            self.optimizer.iterations.numpy(),
                            epoch_i,
                            batch_i,
                            batch_num,
                            loss, (self.ComputeMetrics.result()), rate))
            avg_loss.reset_states()
            self.ComputeMetrics.reset_states()
                        # avg_mae.reset_states()
            start = time.time()
        train_end = time.time()
        print(
                '\nTrain time for epoch #{} ({} total steps): {}'.format(epoch_i + 1, self.optimizer.iterations.numpy(),
                                                                         train_end - train_start))
            #             with self.test_summary_writer.as_default():
        self.testing((test_X, test_y), self.optimizer.iterations)
            # self.checkpoint.save(self.checkpoint_prefix)
      self.export_path = os.path.join(MODEL_DIR, 'export')
      tf.saved_model.save(self.model, self.export_path)
  def testing(self, test_dataset, step_num):
      test_X, test_y = test_dataset
      test_batches = get_batches(test_X, test_y, self.batch_size)

      """Perform an evaluation of `model` on the examples from `dataset`."""
      avg_loss = tf.keras.metrics.Mean('loss', dtype=tf.float32)
        #         avg_mae = tf.keras.metrics.Mean('mae', dtype=tf.float32)

      batch_num = (len(test_X) // self.batch_size)
      for batch_i in range(batch_num):
          x, y = next(test_batches)
          categories = np.zeros([self.batch_size, 18])
          for i in range(self.batch_size):
              categories[i] = x.take(6, 1)[i]

          titles = np.zeros([self.batch_size, sentences_size])
          for i in range(self.batch_size):
              titles[i] = x.take(5, 1)[i]

          logits = self.model([np.reshape(x.take(0, 1), [self.batch_size, 1]).astype(np.float32),
                                 np.reshape(x.take(2, 1), [self.batch_size, 1]).astype(np.float32),
                                 np.reshape(x.take(3, 1), [self.batch_size, 1]).astype(np.float32),
                                 np.reshape(x.take(4, 1), [self.batch_size, 1]).astype(np.float32),
                                 np.reshape(x.take(1, 1), [self.batch_size, 1]).astype(np.float32),
                                 categories.astype(np.float32),
                                 titles.astype(np.float32)], training=False)
          test_loss = self.ComputeLoss(np.reshape(y, [self.batch_size, 1]).astype(np.float32), logits)
          avg_loss(test_loss)
            # 保存测试损失
          self.losses['test'].append(test_loss)
          self.ComputeMetrics(np.reshape(y, [self.batch_size, 1]).astype(np.float32), logits)
            # avg_loss(self.compute_loss(labels, logits))
            # avg_mae(self.compute_metrics(labels, logits))

      print('Model test set loss: {:0.6f} mae: {:0.6f}'.format(avg_loss.result(), self.ComputeMetrics.result()))
        # print('Model test set loss: {:0.6f} mae: {:0.6f}'.format(avg_loss.result(), avg_mae.result()))
        #         summary_ops_v2.scalar('loss', avg_loss.result(), step=step_num)
        #         summary_ops_v2.scalar('mae', self.ComputeMetrics.result(), step=step_num)
        # summary_ops_v2.scalar('mae', avg_mae.result(), step=step_num)

      if avg_loss.result() < self.best_loss:
          self.best_loss = avg_loss.result()
          print("best loss = {}".format(self.best_loss))
          self.checkpoint.save(self.checkpoint_prefix)

  def forward(self, xs):
      predictions = self.model(xs)
        # logits = tf.nn.softmax(predictions)

      return predictions
  



    





使用keras.layers.Reshape实现keras不同层的对接,改变了数据的维度

In [ ]:
mv_net=mv_network()
mv_net.training(features,targets_values,epochs=5)

KerasTensor(type_spec=TensorSpec(shape=(None, 1, 32), dtype=tf.float32, name=None), name='uid_embed_layer/embedding_lookup/Identity_1:0', description="created by layer 'uid_embed_layer'")
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_titles (InputLayer)       [(None, 15)]         0                                            
__________________________________________________________________________________________________
movie_title_embed_layer (Embedd (None, 15, 32)       166880      movie_titles[0][0]               
__________________________________________________________________________________________________
reshape_15 (Reshape)            (None, 15, 32, 1)    0           movie_title_embed_layer[0][0]    
__________________________________________________________________________________________________
con

train_step:优化的是loss值，解决的其实是一个极小化loss问题。这里的含义是使用Adam下降算法（在tensorflow中已经写好了各种优化算法，这里只需要声明和调用即可），使loss值最小，也就是使网络的输出与样本的输出接近。这里的Loss损失函数，可以是均方误差，自定义函数或者交叉熵。train_step在后面调用sess.run()会话计算时，会喂入输入数据。每喂入一组，就计算一次会话，更新一轮参数，所以train_step的含义我理解应该是每次喂入训练数据后执行的结果，可以翻译成“训练步骤”。

Tensorflow的Checkpoint机制将可追踪变量以二进制的方式储存成一个.ckpt文件，储存了变量的名称及对应张量的值。

  Checkpoint 只保存模型的参数，不保存模型的计算过程，因此一般用于在具有模型源代码的时候恢复之前训练好的模型参数。如果需要导出模型（无需源代码也能运行模型）。

